In [1]:
import pandas as pd
import numpy as np
from math import sqrt
import pickle
import requests
from pandas import json_normalize
import json

#from io import BytesIO
#from io import StringIO
#mport boto3

import matplotlib.pyplot as plt
import seaborn as sns
import time
from datetime import date
from datetime import datetime
import datetime
random_state = 5

import urllib.request
from html_table_parser.parser import HTMLTableParser
from pprint import pprint

from sklearn.linear_model import LinearRegression, Lasso
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, RandomForestClassifier
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score, confusion_matrix
import shap

from statsmodels.stats.outliers_influence import variance_inflation_factor

#adjust view
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 10)
pd.set_option('display.width', 1000)

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


## 1.) pull previous weeks game file

In [3]:
previous_weeks_games = pd.read_csv('predictions/previous_week_games.csv')
# previous_weeks_games = previous_weeks_games.drop(columns = ['Unnamed: 0'])

previous_weeks_games_list = previous_weeks_games['id']
previous_weeks_games_list

0     401547609
1     401547610
2     401547611
3     401547612
4     401547613
5     401547614
6     401547616
7     401547617
8     401547618
9     401547615
10    401547619
11    401547620
12    401547621
13    401547351
14    401547241
15    401547622
Name: id, dtype: int64

### 1.a ) scrape stats for previous weeks games

In [5]:
# Opens a website and read its
# binary contents (HTTP Response Body)
def url_get_contents(url):

    # Opens a website and read its
    # binary contents (HTTP Response Body)

    #making request to the website
    req = urllib.request.Request(url=url)
    f = urllib.request.urlopen(req)

    #reading contents of the website
    return f.read()

In [6]:
game_stats_big_df = pd.DataFrame()

for i in previous_weeks_games_list:
    
    try:
        web = "https://www.espn.com/nfl/matchup?gameId=" + str(i)

        # defining the html contents of a URL.
        xhtml = url_get_contents(web).decode('utf-8')

        # Defining the HTMLTableParser object
        p = HTMLTableParser()

        # feeding the html contents in the
        # HTMLTableParser object
        p.feed(xhtml)

        # Now finally obtaining the data of
        # the table required
        df_testing = pd.DataFrame(p.tables[1])
        column_list = list(df_testing[0].values)

        away = pd.DataFrame(df_testing[1]).T
        away.columns = column_list
        away = away.reset_index(drop=True)
        # away = away.drop(columns = ['index'])
        #away.drop(away.columns[[13]], axis=1, inplace=True)
        away = away.rename(columns = {'1st Downs': 'first_downs_away',
                            'Passing 1st downs': 'first_downs_passing_away',
                            'Rushing 1st downs': 'first_downs_rushing_away', 
                            '1st downs from penalties': 'first_down_by_penalty_away', 
                            '3rd down efficiency': 'third_down_away', 
                            '4th down efficiency': 'fourth_down_away', 
                            'Total Plays': 'total_plays_away', 
                            'Total Yards': 'total_yards_away', 
                            'Total Drives': 'total_drives_away', 
                            'Yards per Play': 'yards_per_play_away', 
                            'Passing': 'passing_away', 
                            'Comp-Att': 'comp_att_away', 
                            'Yards per pass': 'yards_per_pass_away',
                            'Fumbles lost': 'fumbles_lost_away',  
                            'Sacks-Yards Lost': 'sack_yards_lost_away', 
                            'Rushing': 'rushing_away', 
                            'Rushing Attempts': 'rushing_att_away', 
                            'Yards per rush': 'yards_per_rush_away', 
                            'Red Zone (Made-Att)': 'red_zone_att_away', 
                            'Penalties': 'penalty_away', 
                            'Turnovers': 'turnovers_away', 
                            'Interceptions thrown': 'int_thrown_away', 
                            'Rushing 1st downs': 'first_down_rushing_away', 
                            'Defensive / Special Teams TDs': 'defensive_td_away', 
                            'Possession': 'possession_away'})

        home = pd.DataFrame(df_testing[2]).T
        home.columns = column_list
        home = home.reset_index(drop=True)
        # home = home.drop(columns = ['index'])
        #home.drop(home.columns[[13]], axis=1, inplace=True)
        home = home.rename(columns = {'1st Downs': 'first_downs_home',
                            'Passing 1st downs': 'first_downs_passing_home',
                            'Rushing 1st downs': 'first_downs_rushing_home', 
                            '1st downs from penalties': 'first_down_by_penalty_home', 
                            '3rd down efficiency': 'third_down_home', 
                            '4th down efficiency': 'fourth_down_home', 
                            'Total Plays': 'total_plays_home', 
                            'Total Yards': 'total_yards_home', 
                            'Total Drives': 'total_drives_home', 
                            'Yards per Play': 'yards_per_play_home', 
                            'Passing': 'passing_home', 
                            'Comp-Att': 'comp_att_home', 
                            'Yards per pass': 'yards_per_pass_home',
                            'Fumbles lost': 'fumbles_lost_home',  
                            'Sacks-Yards Lost': 'sack_yards_lost_home', 
                            'Rushing': 'rushing_home', 
                            'Rushing Attempts': 'rushing_att_home', 
                            'Yards per rush': 'yards_per_rush_home', 
                            'Red Zone (Made-Att)': 'red_zone_att_home', 
                            'Penalties': 'penalty_home', 
                            'Turnovers': 'turnovers_home', 
                            'Interceptions thrown': 'int_thrown_home', 
                            'Rushing 1st downs': 'first_down_rushing_home', 
                            'Defensive / Special Teams TDs': 'defensive_td_home', 
                            'Possession': 'possession_home'})

        game_stat_temp = pd.concat([away, home], axis=1)
        game_stat_temp['id'] = i
        # game_stats_big_df = game_stats_big_df.append(game_stat_temp)
        game_stats_big_df = pd.concat([game_stats_big_df, game_stat_temp], axis=0).reset_index(drop=True)

    except Exception as e:
        print("Error:", e)

In [9]:
try:
    game_stats_big_df = game_stats_big_df.drop(columns = [''])
except:
    pass

In [11]:
# game_stats_big_df.columns

### 1.b ) scrape game results for previous weeks games

In [12]:
game_score_big_df = pd.DataFrame()

for i in previous_weeks_games_list:
    
    try:
        # defining the html contents of a URL.
        web = "https://www.espn.com/nfl/matchup?gameId=" + str(i)

        xhtml = url_get_contents(web).decode('utf-8')

        # Defining the HTMLTableParser object
        p = HTMLTableParser()

        # feeding the html contents in the
        # HTMLTableParser object
        p.feed(xhtml)

        df_score = pd.DataFrame(p.tables[0])
        df_score = pd.DataFrame(df_score.iloc[1: , :])
        team_column = df_score[0].to_frame()
        #team_column = team_column.rename(columns = {0: 'team'})
        #team_df = pd.DataFrame(team_column, columns = ['team'])
        score_column = df_score.iloc[: , -1].to_frame()
        #score_column = score_column.rename({: 'score'})
        #score_df = pd.DataFrame(score_column, columns = ['score'])
        df_score_new = pd.concat([team_column, score_column], axis=1)
        df_score_new.columns = ['team', 'score']

        away_score = pd.DataFrame(df_score_new.iloc[0]).T
        away_score = away_score.reset_index(drop=True)
        # away_score = away_score.drop(columns = ['index'])
        #away_score = away_score[['team', 'score']]
        away_score = away_score.rename(columns = {'team': 'away_team', 'score': 'away_score'})

        home_score = pd.DataFrame(df_score_new.iloc[1]).T
        home_score = home_score.reset_index(drop=True)
        # home_score = home_score.drop(columns = ['index'])
        #home_score = home_score[['team', 'score']]
        home_score = home_score.rename(columns = {'team': 'home_team', 'score': 'home_score'})

        game_score_temp = pd.concat([away_score, home_score], axis=1)
        game_score_temp['id'] = i
        #game_score_temp = game_score_temp.reset_index()
        #game_score_temp = game_score_temp.drop(columns = ['index'])
        # game_score_big_df = game_score_big_df.append(game_score_temp)
        game_score_big_df = pd.concat([game_score_big_df, game_score_temp], axis=0).reset_index(drop=True)

    except Exception as e:
        print('Error:', e, 'game_id=', i)

In [13]:
game_score_big_df

,away_team,away_score,home_team,home_score,id
0,NO,22,LAR,30,401547609
1,CIN,11,PIT,34,401547610
2,BUF,24,LAC,22,401547611
3,CLE,36,HOU,22,401547612
4,WSH,28,NYJ,30,401547613
5,SEA,20,TEN,17,401547614
6,IND,10,ATL,29,401547616
7,GB,33,CAR,30,401547617
8,DET,30,MIN,24,401547618
9,JAX,12,TB,30,401547615


### 1.c ) join these three (previous_weeks_games_df, game_stats_df, and game_score_df) data sets

In [14]:
game_stats_big_df.columns

Index(['first_downs_away', 'first_downs_passing_away', 'first_down_rushing_away', 'first_down_by_penalty_away', 'third_down_away', 'fourth_down_away', 'total_plays_away', 'total_yards_away', 'total_drives_away', 'yards_per_play_away', 'passing_away', 'comp_att_away', 'yards_per_pass_away', 'int_thrown_away', 'sack_yards_lost_away', 'rushing_away', 'rushing_att_away', 'yards_per_rush_away', 'red_zone_att_away', 'penalty_away', 'turnovers_away', 'fumbles_lost_away', 'int_thrown_away', 'defensive_td_away', 'possession_away', 'first_downs_home', 'first_downs_passing_home', 'first_down_rushing_home', 'first_down_by_penalty_home', 'third_down_home', 'fourth_down_home', 'total_plays_home', 'total_yards_home', 'total_drives_home', 'yards_per_play_home', 'passing_home', 'comp_att_home', 'yards_per_pass_home', 'int_thrown_home', 'sack_yards_lost_home', 'rushing_home', 'rushing_att_home', 'yards_per_rush_home', 'red_zone_att_home', 'penalty_home', 'turnovers_home', 'fumbles_lost_home',
       'in

In [15]:
df_ready_to_clean = game_stats_big_df.merge(game_score_big_df, on='id', how='inner')
df_ready_to_clean = df_ready_to_clean.merge(previous_weeks_games, on='id', how='inner')
df_ready_to_clean.head()

,first_downs_away,first_downs_passing_away,first_down_rushing_away,first_down_by_penalty_away,third_down_away,...,away_team,away_score,home_team,home_score,url
0,18,16,2,0,5-11,...,NO,22,LAR,30,https://www.espn.com/nfl/scoreboard/_/week/16/...
1,17,13,3,1,6-13,...,CIN,11,PIT,34,https://www.espn.com/nfl/scoreboard/_/week/16/...
2,16,11,5,0,5-10,...,BUF,24,LAC,22,https://www.espn.com/nfl/scoreboard/_/week/16/...
3,26,19,4,3,6-16,...,CLE,36,HOU,22,https://www.espn.com/nfl/scoreboard/_/week/16/...
4,18,7,7,4,3-12,...,WSH,28,NYJ,30,https://www.espn.com/nfl/scoreboard/_/week/16/...


In [16]:
df_ready_to_clean.columns

Index(['first_downs_away', 'first_downs_passing_away', 'first_down_rushing_away', 'first_down_by_penalty_away', 'third_down_away', 'fourth_down_away', 'total_plays_away', 'total_yards_away', 'total_drives_away', 'yards_per_play_away', 'passing_away', 'comp_att_away', 'yards_per_pass_away', 'int_thrown_away', 'sack_yards_lost_away', 'rushing_away', 'rushing_att_away', 'yards_per_rush_away', 'red_zone_att_away', 'penalty_away', 'turnovers_away', 'fumbles_lost_away', 'int_thrown_away', 'defensive_td_away', 'possession_away', 'first_downs_home', 'first_downs_passing_home', 'first_down_rushing_home', 'first_down_by_penalty_home', 'third_down_home', 'fourth_down_home', 'total_plays_home', 'total_yards_home', 'total_drives_home', 'yards_per_play_home', 'passing_home', 'comp_att_home', 'yards_per_pass_home', 'int_thrown_home', 'sack_yards_lost_home', 'rushing_home', 'rushing_att_home', 'yards_per_rush_home', 'red_zone_att_home', 'penalty_home', 'turnovers_home', 'fumbles_lost_home',
       'in

## 2.) Begin Cleaning Stats data

In [17]:
def convert_fraction(df, column_name, new_column_name):
    # new data frame with split value columns
    new = df[column_name].str.split("-", n = 1, expand = True)
  
    # making separate first name column from new data frame
    df["numerator"]= new[0]
    df["numerator"] = df["numerator"].astype(int)
  
    # making separate last name column from new data frame
    df["denominator"]= new[1]
    df["denominator"] = df["denominator"].astype(int)

    df[new_column_name] = df["numerator"]/df["denominator"]

    # Dropping old Name columns
    df.drop(columns =[column_name, 'numerator', 'denominator'], inplace = True)
    return df

In [18]:
def convert_att_yards(df, column_name):
    # new data frame with split value columns
    new = df[column_name].str.split("-", n = 1, expand = True)
  
    # making separate first name column from new data frame
    att_col_str = column_name + "_occur"
    yards_col_str = column_name + "_yards"

    df[att_col_str]= new[0]
    df[att_col_str] = df[att_col_str].astype(int)
  
    # making separate last name column from new data frame
    df[yards_col_str]= new[1]
    df[yards_col_str] = df[yards_col_str].astype(int)

    # Dropping old Name columns
    df.drop(columns =[column_name], inplace = True)
    return df

In [19]:
df_ready_to_clean = convert_fraction(df_ready_to_clean, "third_down_away", "third_down_away")
df_ready_to_clean = convert_fraction(df_ready_to_clean, "third_down_home", "third_down_home")
df_ready_to_clean = convert_fraction(df_ready_to_clean, "fourth_down_away", "fourth_down_away")
df_ready_to_clean = convert_fraction(df_ready_to_clean, "fourth_down_home", "fourth_down_home")
df_ready_to_clean = convert_fraction(df_ready_to_clean, "comp_att_away", "comp_att_away")
df_ready_to_clean = convert_fraction(df_ready_to_clean, "comp_att_home", "comp_att_home")
df_ready_to_clean = df_ready_to_clean.fillna(0)

## need to clean a few other variables that are being treated as str's
df_ready_to_clean = convert_att_yards(df_ready_to_clean, "sack_yards_lost_away")
df_ready_to_clean = convert_att_yards(df_ready_to_clean, "sack_yards_lost_home")
df_ready_to_clean = convert_att_yards(df_ready_to_clean, "penalty_away")
df_ready_to_clean = convert_att_yards(df_ready_to_clean, "penalty_home")

## change possession into float
df_ready_to_clean['possession_away'] = df_ready_to_clean['possession_away'].str.replace(':', '.')
df_ready_to_clean['possession_home'] = df_ready_to_clean['possession_home'].str.replace(':', '.')
df_ready_to_clean['possession_away'] = df_ready_to_clean['possession_away'].astype(float)
df_ready_to_clean['possession_home'] = df_ready_to_clean['possession_home'].astype(float)
df_ready_to_clean.head()

,first_downs_away,first_downs_passing_away,first_down_rushing_away,first_down_by_penalty_away,total_plays_away,...,sack_yards_lost_home_yards,penalty_away_occur,penalty_away_yards,penalty_home_occur,penalty_home_yards
0,18,16,2,0,58,...,3,3,20,3,25
1,17,13,3,1,61,...,6,3,20,2,15
2,16,11,5,0,52,...,40,7,64,6,29
3,26,19,4,3,75,...,33,9,55,10,76
4,18,7,7,4,58,...,0,8,68,14,150


In [20]:
## needing to declare the winning team to create target variable
df_ready_to_clean['target'] = np.where(df_ready_to_clean['away_score'] > df_ready_to_clean['home_score'], 1, 0)
df_ready_to_clean['winning_team'] = np.where(df_ready_to_clean['away_score'] > df_ready_to_clean['home_score'], df_ready_to_clean['away_team'], df_ready_to_clean['home_team'])
df_ready_to_clean.head()

,first_downs_away,first_downs_passing_away,first_down_rushing_away,first_down_by_penalty_away,total_plays_away,...,penalty_away_yards,penalty_home_occur,penalty_home_yards,target,winning_team
0,18,16,2,0,58,...,20,3,25,0,LAR
1,17,13,3,1,61,...,20,2,15,0,PIT
2,16,11,5,0,52,...,64,6,29,1,BUF
3,26,19,4,3,75,...,55,10,76,1,CLE
4,18,7,7,4,58,...,68,14,150,0,NYJ


### 2.a ) creating and updating historical stats file

In [21]:
## dropping a few last un-needed cloumns
df_ready_to_clean = df_ready_to_clean.loc[:,~df_ready_to_clean.columns.duplicated()]
df_clean = df_ready_to_clean.drop(columns=['red_zone_att_away', 'red_zone_att_home'])
#df_clean.to_csv('data/nfl_historical_clean.csv')
print(df_clean.groupby('target').size())
df_clean.head()

target
0    8
1    8
dtype: int64


,first_downs_away,first_downs_passing_away,first_down_rushing_away,first_down_by_penalty_away,total_plays_away,...,penalty_away_yards,penalty_home_occur,penalty_home_yards,target,winning_team
0,18,16,2,0,58,...,20,3,25,0,LAR
1,17,13,3,1,61,...,20,2,15,0,PIT
2,16,11,5,0,52,...,64,6,29,1,BUF
3,26,19,4,3,75,...,55,10,76,1,CLE
4,18,7,7,4,58,...,68,14,150,0,NYJ


In [22]:
# df_clean.columns

In [23]:
try:
    df_clean = df_clean.drop(columns=[''])
except:
    pass

In [31]:
df_clean_hist = pd.read_csv('data/nfl_historical_clean.csv')
print(df_clean_hist.shape)
# df_clean_hist = df_clean_hist.drop(columns = 'Unnamed: 0')
# df_clean_hist = df_clean_hist.append(df_clean, ignore_index=True)
df_clean_hist = pd.concat([df_clean_hist, df_clean], axis=0).reset_index(drop=True)
df_clean_hist = df_clean_hist.drop_duplicates(keep='last').reset_index(drop=True)
print(df_clean_hist.shape)
df_clean_hist.tail()

(5862, 52)
(5878, 52)


,first_downs_away,first_downs_passing_away,first_down_rushing_away,first_down_by_penalty_away,total_plays_away,...,penalty_home_occur,penalty_home_yards,target,winning_team,url
5873,19,9,7,3,61,...,5,33,0,MIA,https://www.espn.com/nfl/scoreboard/_/week/16/...
5874,12,11,1,0,56,...,2,25,1,NE,https://www.espn.com/nfl/scoreboard/_/week/16/...
5875,12,3,8,1,52,...,5,44,1,LV,https://www.espn.com/nfl/scoreboard/_/week/16/...
5876,14,7,6,1,59,...,9,65,0,PHI,https://www.espn.com/nfl/scoreboard/_/week/16/...
5877,23,14,4,5,63,...,10,102,1,BAL,https://www.espn.com/nfl/scoreboard/_/week/16/...


In [32]:
df_clean_hist.columns

Index(['first_downs_away', 'first_downs_passing_away', 'first_down_rushing_away', 'first_down_by_penalty_away', 'total_plays_away', 'total_yards_away', 'total_drives_away', 'yards_per_play_away', 'passing_away', 'yards_per_pass_away', 'int_thrown_away', 'rushing_away', 'rushing_att_away', 'yards_per_rush_away', 'turnovers_away', 'fumbles_lost_away', 'defensive_td_away', 'possession_away', 'first_downs_home', 'first_downs_passing_home', 'first_down_rushing_home', 'first_down_by_penalty_home', 'total_plays_home', 'total_yards_home', 'total_drives_home', 'yards_per_play_home', 'passing_home', 'yards_per_pass_home', 'int_thrown_home', 'rushing_home', 'rushing_att_home', 'yards_per_rush_home', 'turnovers_home', 'fumbles_lost_home', 'defensive_td_home', 'possession_home', 'id', 'away_team', 'away_score', 'home_team', 'home_score', 'sack_yards_lost_away_occur', 'sack_yards_lost_away_yards', 'sack_yards_lost_home_occur', 'sack_yards_lost_home_yards', 'penalty_away_occur',
       'penalty_away_

In [33]:
df_clean_hist.to_csv('data/nfl_historical_clean.csv', index=False)
df_clean_hist.shape

(5878, 52)

### 2.b ) creating and updating df_ready_for_model file

In [34]:
df_ready_for_model = df_ready_to_clean.drop(columns=['red_zone_att_away', 'red_zone_att_home', 'url', 'winning_team', 'away_team', 'away_score', 'home_team', 'home_score', 'id'])
#df_ready_for_model.to_csv('data/df_ready_for_model.csv')
df_ready_for_model.head()

,first_downs_away,first_downs_passing_away,first_down_rushing_away,first_down_by_penalty_away,total_plays_away,...,penalty_away_occur,penalty_away_yards,penalty_home_occur,penalty_home_yards,target
0,18,16,2,0,58,...,3,20,3,25,0
1,17,13,3,1,61,...,3,20,2,15,0
2,16,11,5,0,52,...,7,64,6,29,1
3,26,19,4,3,75,...,9,55,10,76,1
4,18,7,7,4,58,...,8,68,14,150,0


In [35]:
df_ready_for_model.shape

(16, 45)

In [78]:
df_ready_for_model.columns

Index(['', 'first_downs_away', 'first_downs_passing_away', 'first_down_rushing_away', 'first_down_by_penalty_away', 'total_plays_away', 'total_yards_away', 'total_drives_away', 'yards_per_play_away', 'passing_away', 'yards_per_pass_away', 'int_thrown_away', 'rushing_away', 'rushing_att_away', 'yards_per_rush_away', 'turnovers_away', 'fumbles_lost_away', 'defensive_td_away', 'possession_away', 'first_downs_home', 'first_downs_passing_home', 'first_down_rushing_home', 'first_down_by_penalty_home', 'total_plays_home', 'total_yards_home', 'total_drives_home', 'yards_per_play_home', 'passing_home', 'yards_per_pass_home', 'int_thrown_home', 'rushing_home', 'rushing_att_home', 'yards_per_rush_home', 'turnovers_home', 'fumbles_lost_home', 'defensive_td_home', 'possession_home', 'sack_yards_lost_away_occur', 'sack_yards_lost_away_yards', 'sack_yards_lost_home_occur', 'sack_yards_lost_home_yards', 'penalty_away_occur', 'penalty_away_yards', 'penalty_home_occur', 'penalty_home_yards', 'target'], 

In [36]:
df_ready_for_model_hist = pd.read_csv('data/df_ready_for_model.csv')
print(df_ready_for_model_hist.shape)
# df_ready_for_model_hist = df_ready_for_model_hist.drop(columns = 'Unnamed: 0')
# df_ready_for_model_hist = df_ready_for_model_hist.append(df_ready_for_model, ignore_index=True)
df_ready_for_model_hist = pd.concat([df_ready_for_model_hist, df_ready_for_model], axis=0).reset_index(drop=True)
print(df_ready_for_model_hist.shape)
df_ready_for_model_hist.tail()

(6189, 45)
(6205, 45)


,first_downs_away,first_downs_passing_away,first_down_rushing_away,first_down_by_penalty_away,total_plays_away,...,penalty_away_occur,penalty_away_yards,penalty_home_occur,penalty_home_yards,target
6200,19,9,7,3,61,...,6,38,5,33,0
6201,12,11,1,0,56,...,3,25,2,25,1
6202,12,3,8,1,52,...,4,20,5,44,1
6203,14,7,6,1,59,...,3,13,9,65,0
6204,23,14,4,5,63,...,6,67,10,102,1


In [37]:
df_ready_for_model_hist.to_csv('data/df_ready_for_model.csv', index=False)
df_ready_for_model_hist.shape

(6205, 45)

## 3.) Retraining Model and Update

In [38]:
df_ready_for_model_hist = pd.read_csv('data/df_ready_for_model.csv')
# df_ready_for_model_hist = df_ready_for_model_hist.drop(columns = 'Unnamed: 0')
df_model = df_ready_for_model_hist
df_model['target_class']  = np.where(df_model['target'] == 1, "away", "home")
print(df_model.groupby('target_class').size())
df_model = df_model.drop(columns = ['target'])
# df_model = df_model.drop_duplicates()
df_model.head()

target_class
away    2705
home    3500
dtype: int64


,first_downs_away,first_downs_passing_away,first_down_rushing_away,first_down_by_penalty_away,total_plays_away,...,penalty_away_occur,penalty_away_yards,penalty_home_occur,penalty_home_yards,target_class
0,13,7,5,1,51,...,5,29,10,80,away
1,18,11,4,3,47,...,10,90,10,82,away
2,19,9,9,1,62,...,8,52,4,33,home
3,27,9,16,2,65,...,4,39,9,57,away
4,24,12,10,2,59,...,9,87,4,38,away


In [39]:
data_bucket = df_model.copy()
data_bucket = data_bucket.dropna()
data_len = len(data_bucket.columns)

y_adj_bucket = data_bucket.iloc[:,data_len-1:]
X_adj_bucket = data_bucket.iloc[:,:data_len-1]

In [40]:
rf_clf =RandomForestClassifier(max_depth = 10,n_estimators=40, max_features = 'log2', random_state=0)
#rf_clf =RandomForestClassifier(max_depth = 6,n_estimators=10, max_features = 'log2', random_state=0, class_weight={'0-4%': 1, '4-7.6%': 2})
np.random.seed(42)

In [41]:
##smote
sm = SMOTE(random_state=42)

## training
#X_train_res, y_train_res = sm.fit_resample(x_train, y_train)

## productionize
X_res, y_res = sm.fit_resample(X_adj_bucket, y_adj_bucket)

##random oversample
#ros = RandomOverSampler(random_state=42)
#X_train_res, y_train_res = ros.fit_resample(X_train, y_train)

classifier = rf_clf.fit(X_res, y_res)
#classifier = rf_clf.fit(x_train, y_train)
#print("Model Score is:", rf_clf.score(x_test, y_test))
#y_pred_train = rf_clf.predict(X_train_res)
y_pred = rf_clf.predict(X_res)
#y_pred_test = rf_clf.predict(x_test)

## saving model
# save the model to disk
filename = 'models/random_forest_model_1.sav'
pickle.dump(classifier, open(filename, 'wb'))

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


### extract shap values to use in app

In [42]:
# compute SHAP values
explainer = shap.TreeExplainer(rf_clf, X_adj_bucket, check_additivity=False)
shap_values = explainer.shap_values(X_adj_bucket, check_additivity=False)

100%|===================| 12370/12410 [01:35<00:00]        

In [46]:
shap_values_class_0 = pd.DataFrame(shap_values[0])
shap_values_class_1 = pd.DataFrame(shap_values[1])

shap_values_class_0.to_csv('shap_values/shap_values_class_0.csv')
shap_values_class_1.to_csv('shap_values/shap_values_class_1.csv')

## 4.) Prepare teams historical stats file and update

In [47]:
stats_agg_df = pd.read_csv('data/nfl_historical_clean.csv')
# stats_agg_df = stats_agg_df.drop(columns = 'Unnamed: 0')
# stats_agg_df = stats_agg_df.drop_duplicates()

stats_agg_df['year'] = stats_agg_df['url'].str[-17:-13]
stats_agg_df['year'] = stats_agg_df['year'].astype(int)
today = date.today()
year = today.year
stats_agg_df = stats_agg_df[(stats_agg_df['year'] == year) | (stats_agg_df['year'] == year-1)]
stats_agg_df.tail()

,first_downs_away,first_downs_passing_away,first_down_rushing_away,first_down_by_penalty_away,total_plays_away,...,penalty_home_yards,target,winning_team,url,year
5873,19,9,7,3,61,...,33,0,MIA,https://www.espn.com/nfl/scoreboard/_/week/16/...,2023
5874,12,11,1,0,56,...,25,1,NE,https://www.espn.com/nfl/scoreboard/_/week/16/...,2023
5875,12,3,8,1,52,...,44,1,LV,https://www.espn.com/nfl/scoreboard/_/week/16/...,2023
5876,14,7,6,1,59,...,65,0,PHI,https://www.espn.com/nfl/scoreboard/_/week/16/...,2023
5877,23,14,4,5,63,...,102,1,BAL,https://www.espn.com/nfl/scoreboard/_/week/16/...,2023


In [48]:
team_list = stats_agg_df['away_team'].unique()
len(team_list)

32

In [50]:
# stat_agg_df.head()

In [53]:
stat_agg_df = pd.DataFrame()

for i in team_list:
    team = i

    new_DataFrame = stats_agg_df[(stats_agg_df['away_team'] == team) | (stats_agg_df['home_team'] == team)]

    stat_df_away = new_DataFrame[new_DataFrame['away_team'] == team]
    stat_df_away = stat_df_away[['first_downs_away', 'first_downs_passing_away', 'first_down_rushing_away', 'first_down_by_penalty_away', 'total_plays_away', 'total_yards_away', 'total_drives_away', 'yards_per_play_away', 'passing_away', 'yards_per_pass_away', 'int_thrown_away', 'rushing_away', 'rushing_att_away', 'yards_per_rush_away', 'turnovers_away', 'fumbles_lost_away', 'defensive_td_away', 'possession_away', 'sack_yards_lost_away_occur', 'sack_yards_lost_away_yards', 'penalty_away_occur', 'penalty_away_yards']]
    stat_df_away = stat_df_away.rename(columns = {'first_downs_away': 'first_downs', 'first_downs_passing_away': 'first_downs_passing', 'first_down_rushing_away': 'first_down_rushing', 'first_down_by_penalty_away': 'first_down_by_penalty', 'total_plays_away': 'total_plays', 'total_yards_away': 'total_yards', 'total_drives_away': 'total_drives', 'yards_per_play_away': 'yards_per_play', 'passing_away': 'passing', 'yards_per_pass_away': 'yards_per_pass', 'int_thrown_away': 'int_thrown', 'rushing_away': 'rushing', 'rushing_att_away': 'rushing_att', 'yards_per_rush_away': 'yards_per_rush', 'turnovers_away': 'turnovers', 'fumbles_lost_away': 'fumbles_lost', 'defensive_td_away': 'defensive_td', 'possession_away': 'possession', 'sack_yards_lost_away_occur': 'sack_yards_lost_occur', 'sack_yards_lost_away_yards': 'sack_yards_lost_yards', 'penalty_away_occur': 'penalty_occur', 'penalty_away_yards': 'penalty_yards'})
    
    stat_df_home = new_DataFrame[new_DataFrame['home_team'] == team]
    stat_df_home = stat_df_home[['first_downs_home', 'first_downs_passing_home', 'first_down_rushing_home', 'first_down_by_penalty_home', 'total_plays_home', 'total_yards_home', 'total_drives_home', 'yards_per_play_home', 'passing_home', 'yards_per_pass_home', 'int_thrown_home', 'rushing_home', 'rushing_att_home', 'yards_per_rush_home', 'turnovers_home', 'fumbles_lost_home', 'defensive_td_home', 'possession_home', 'sack_yards_lost_home_occur', 'sack_yards_lost_home_yards', 'penalty_home_occur', 'penalty_home_yards']]
    stat_df_home = stat_df_home.rename(columns = {'first_downs_home': 'first_downs', 'first_downs_passing_home': 'first_downs_passing', 'first_down_rushing_home': 'first_down_rushing', 'first_down_by_penalty_home': 'first_down_by_penalty', 'total_plays_home': 'total_plays', 'total_yards_home': 'total_yards', 'total_drives_home': 'total_drives', 'yards_per_play_home': 'yards_per_play', 'passing_home': 'passing', 'yards_per_pass_home': 'yards_per_pass', 'int_thrown_home': 'int_thrown', 'rushing_home': 'rushing', 'rushing_att_home': 'rushing_att', 'yards_per_rush_home': 'yards_per_rush', 'turnovers_home': 'turnovers', 'fumbles_lost_home': 'fumbles_lost', 'defensive_td_home': 'defensive_td', 'possession_home': 'possession', 'sack_yards_lost_home_occur': 'sack_yards_lost_occur', 'sack_yards_lost_home_yards': 'sack_yards_lost_yards', 'penalty_home_occur': 'penalty_occur', 'penalty_home_yards': 'penalty_yards'})

    # team_stats_all = stat_df_away.append(stat_df_home)
    team_stats_all = pd.concat([stat_df_away, stat_df_home], axis=0).reset_index(drop=True)
    team_stats_agg_temp = pd.DataFrame(team_stats_all.mean()).T
    team_stats_agg_temp['team'] = team
    # stat_agg_df = stat_agg_df.append(team_stats_agg_temp)
    stat_agg_df = pd.concat([stat_agg_df, team_stats_agg_temp], axis=0).reset_index(drop=True)
    print(i, "Done")

print("All Done")

BUF Done
NO Done
SF Done
PIT Done
PHI Done
NE Done
BAL Done
JAX Done
CLE Done
IND Done
NYG Done
GB Done
KC Done
LV Done
TB Done
DEN Done
LAC Done
NYJ Done
WSH Done
CAR Done
MIA Done
ATL Done
SEA Done
CIN Done
HOU Done
ARI Done
CHI Done
TEN Done
MIN Done
DET Done
LAR Done
DAL Done
All Done


In [54]:
stat_agg_df.head()

,first_downs,first_downs_passing,first_down_rushing,first_down_by_penalty,total_plays,...,sack_yards_lost_occur,sack_yards_lost_yards,penalty_occur,penalty_yards,team
0,22.441176,12.647059,8.323529,1.470588,64.500000,...,1.823529,9.735294,6.029412,48.705882,BUF
1,18.617647,11.205882,6.058824,1.352941,62.235294,...,2.117647,15.058824,5.764706,50.558824,NO
2,21.083333,11.527778,7.944444,1.611111,60.694444,...,1.972222,11.972222,5.944444,50.166667,SF
3,18.393939,10.000000,7.000000,1.393939,62.515152,...,2.151515,14.787879,5.333333,43.757576,PIT
4,23.000000,11.485714,9.628571,1.885714,66.771429,...,2.314286,13.342857,5.628571,43.742857,PHI


In [56]:
print(stat_agg_df.shape)

(32, 23)


In [55]:
# stat_agg_df = stat_agg_df.drop_duplicates()
stat_agg_df.to_csv('data/agg_team_stats.csv', index=False)

## 5.) Updating all Historical Files

In [59]:
game_id_og = pd.read_csv('data/game_id_all.csv')
# game_id_og = game_id_og.drop(columns = ['Unnamed: 0'])
previous_weeks_games_df = previous_weeks_games
# game_id_all = game_id_og.append(previous_weeks_games_df)
game_id_all = pd.concat([game_id_og, previous_weeks_games_df], axis=0).reset_index(drop=True)
# game_id_all = game_id_all.drop_duplicates()
game_id_all = game_id_all.dropna()
game_id_all

,id,url
0,220905019,https://www.espn.com/nfl/scoreboard/_/week/1/y...
1,220908002,https://www.espn.com/nfl/scoreboard/_/week/1/y...
2,220908003,https://www.espn.com/nfl/scoreboard/_/week/1/y...
3,220908004,https://www.espn.com/nfl/scoreboard/_/week/1/y...
4,220908005,https://www.espn.com/nfl/scoreboard/_/week/1/y...
...,...,...
6088,401547620,https://www.espn.com/nfl/scoreboard/_/week/16/...
6089,401547621,https://www.espn.com/nfl/scoreboard/_/week/16/...
6090,401547351,https://www.espn.com/nfl/scoreboard/_/week/16/...
6091,401547241,https://www.espn.com/nfl/scoreboard/_/week/16/...


In [60]:
game_scores_og = pd.read_csv('data/game_scores.csv')
# game_scores_og = game_scores_og.drop(columns = ['Unnamed: 0'])
# game_scores_all = game_scores_og.append(game_score_big_df)
game_scores_all = pd.concat([game_scores_og, game_score_big_df], axis=0).reset_index(drop=True)
# game_scores_all = game_scores_all.drop_duplicates()
game_scores_all

,away_team,away_score,home_team,home_score,id
0,SF,16,NYG,13,220905019
1,NYJ,37,BUF,31,220908002
2,MIN,23,CHI,27,220908003
3,SD,34,CIN,6,220908004
4,KC,40,CLE,39,220908005
...,...,...,...,...,...
5880,DAL,20,MIA,22,401547620
5881,NE,26,DEN,23,401547621
5882,LV,20,KC,14,401547351
5883,NYG,25,PHI,33,401547241


In [61]:
game_scores_all.to_csv('data/game_scores.csv', index=False)

## 6.) Create previous weeks games file

In [62]:
historical = pd.read_csv('data/agg_team_stats.csv')
probs_historical = pd.read_csv('predictions/model_prediction_file.csv')
# probs_historical = pd.DataFrame(columns = ['away', 'home', 'id'])
# historical = historical.drop(columns='Unnamed: 0')
# probs_historical = probs_historical.drop(columns='Unnamed: 0')

## declaring model name that I would like to use
filename = 'models/random_forest_model_1.sav'
# load the model from disk
model_obj = pickle.load(open(filename, 'rb'))

In [66]:
probs_historical.head()

,away,home,id
0,0.723230,0.276770,401547609
1,0.417934,0.582066,401547610
2,0.747199,0.252801,401547611
3,0.494441,0.505559,401547612
4,0.810684,0.189316,401547613


In [67]:
def get_current_games():
    url = "https://site.web.api.espn.com/apis/v2/scoreboard/header?sport=football&league=nfl&region=us&lang=en&contentorigin=espn&buyWindow=1m&showAirings=buy%2Clive%2Creplay&showZipLookup=true&tz=America/New_York"

    payload={}
    headers = {
    'Accept': 'application/json',
    'Cookie': 'SWID=3750444F-BA06-4929-C99C-62333D7EE8A0'
    }

    response = requests.request("GET", url, headers=headers, data=payload)

    response_text = json.loads(response.text)

    game_df = json_normalize(response_text['sports'], ['leagues', 'events'])
    game_dict = game_df.to_dict('index')
    return game_df, game_dict

In [68]:
game_df, game_dict = get_current_games()
game_df[['away','home']] = game_df.shortName.apply(lambda x: pd.Series(str(x).split("@")))
game_df['away'] = game_df['away'].str.strip()
game_df['home'] = game_df['home'].str.strip()
game_df[['date','time']] = game_df.date.apply(lambda x: pd.Series(str(x).split("T")))
game_df['time'] = game_df['time'].str[:5]
game_df['time'] = game_df['time'].str.replace("18", "12")
game_df['time'] = game_df['time'].str.replace("21", "03")
game_df['time'] = game_df['time'].str.replace("01", "07")
game_df

,id,uid,date,timeValid,recent,...,fullStatus.type.shortDetail,airings,away,home,time
0,401547623,s:20~l:28~e:401547623~c:401547623,2023-12-29,True,False,...,12/28 - 8:15 PM EST,NaN,NYJ,CLE,07:15
1,401547624,s:20~l:28~e:401547624~c:401547624,2023-12-31,True,False,...,12/30 - 8:15 PM EST,"[{'createdOn': '2023-11-27T21:03:24.378+0000',...",DET,DAL,07:15
2,401547625,s:20~l:28~e:401547625~c:401547625,2023-12-31,True,False,...,12/31 - 1:00 PM EST,NaN,MIA,BAL,12:00
3,401547626,s:20~l:28~e:401547626~c:401547626,2023-12-31,True,False,...,12/31 - 1:00 PM EST,NaN,NE,BUF,12:00
4,401547627,s:20~l:28~e:401547627~c:401547627,2023-12-31,True,False,...,12/31 - 1:00 PM EST,NaN,ATL,CHI,12:00
5,401547628,s:20~l:28~e:401547628~c:401547628,2023-12-31,True,False,...,12/31 - 1:00 PM EST,NaN,LV,IND,12:00
6,401547629,s:20~l:28~e:401547629~c:401547629,2023-12-31,True,False,...,12/31 - 1:00 PM EST,NaN,CAR,JAX,12:00
7,401547631,s:20~l:28~e:401547631~c:401547631,2023-12-31,True,False,...,12/31 - 1:00 PM EST,NaN,TEN,HOU,12:00
8,401547632,s:20~l:28~e:401547632~c:401547632,2023-12-31,True,False,...,12/31 - 1:00 PM EST,NaN,LAR,NYG,12:00
9,401547633,s:20~l:28~e:401547633~c:401547633,2023-12-31,True,False,...,12/31 - 1:00 PM EST,NaN,ARI,PHI,12:00


In [69]:
previous_week_games_file_df = game_df[['id', 'season', 'seasonType', 'week']]
previous_week_games_file_df['season'] = previous_week_games_file_df['season'].astype(str)
previous_week_games_file_df['week'] = previous_week_games_file_df['week'].astype(str)
previous_week_games_file_df['url'] = "https://www.espn.com/nfl/scoreboard/_/week/" + previous_week_games_file_df['week'] + "/year/" + previous_week_games_file_df['season'] + "/seasontype/" + previous_week_games_file_df['seasonType']
previous_week_games_file_df = previous_week_games_file_df.drop(columns = ['season', 'seasonType', 'week'])
previous_week_games_file_df.head()


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,id,url
0,401547623,https://www.espn.com/nfl/scoreboard/_/week/17/...
1,401547624,https://www.espn.com/nfl/scoreboard/_/week/17/...
2,401547625,https://www.espn.com/nfl/scoreboard/_/week/17/...
3,401547626,https://www.espn.com/nfl/scoreboard/_/week/17/...
4,401547627,https://www.espn.com/nfl/scoreboard/_/week/17/...


In [127]:
# previous_week_games_file_df

In [70]:
previous_week_games_file_df.to_csv('predictions/previous_week_games.csv', index=False)

In [130]:
# stats_all

In [71]:
column_order

NameError: name 'column_order' is not defined

In [72]:
probabilities_df = pd.DataFrame()
column_order = X_res.columns

for ind in game_df.index:

    game_id = game_df['id'][ind]
    #id_list.append(game_id)

    away = game_df['away'][ind]
    home = game_df['home'][ind]

    away_stat = game_df[game_df['away'] == away]
    away_stat = pd.DataFrame(away_stat['away'])
    away_stat = away_stat.rename(columns = {'away': 'team'})
    #away_stat.columns = ['team']
    away_stat = away_stat.merge(historical, on='team', how='inner')
    away_stat = away_stat.rename(columns = {'first_downs':'first_downs_away', 'first_downs_passing':'first_downs_passing_away', 'first_down_rushing':'first_down_rushing_away', 'first_down_by_penalty':'first_down_by_penalty_away', 'total_plays':'total_plays_away', 'total_yards':'total_yards_away', 'total_drives':'total_drives_away', 'yards_per_play':'yards_per_play_away', 'passing':'passing_away', 'yards_per_pass':'yards_per_pass_away', 'int_thrown':'int_thrown_away', 'rushing':'rushing_away', 'rushing_att':'rushing_att_away', 'yards_per_rush':'yards_per_rush_away', 'turnovers':'turnovers_away', 'fumbles_lost':'fumbles_lost_away', 'defensive_td':'defensive_td_away', 'possession':'possession_away', 'sack_yards_lost_occur':'sack_yards_lost_away_occur', 'sack_yards_lost_yards':'sack_yards_lost_away_yards', 'penalty_occur':'penalty_away_occur', 'penalty_yards':'penalty_away_yards'})
    away_stat = away_stat.reset_index()
    away_stat = away_stat.drop(columns = ['index', 'team'])

    home_stat = game_df[game_df['home'] == home]
    home_stat = pd.DataFrame(home_stat['home'])
    home_stat = home_stat.rename(columns = {'home': 'team'})
    #home_stat.columns = ['team']
    home_stat = home_stat.merge(historical, on='team', how='inner')
    home_stat = home_stat.rename(columns = {'first_downs':'first_downs_home', 'first_downs_passing':'first_downs_passing_home', 'first_down_rushing':'first_down_rushing_home', 'first_down_by_penalty':'first_down_by_penalty_home', 'total_plays':'total_plays_home', 'total_yards':'total_yards_home', 'total_drives':'total_drives_home', 'yards_per_play':'yards_per_play_home', 'passing':'passing_home', 'yards_per_pass':'yards_per_pass_home', 'int_thrown':'int_thrown_home', 'rushing':'rushing_home', 'rushing_att':'rushing_att_home', 'yards_per_rush':'yards_per_rush_home', 'turnovers':'turnovers_home', 'fumbles_lost':'fumbles_lost_home', 'defensive_td':'defensive_td_home', 'possession':'possession_home', 'sack_yards_lost_occur':'sack_yards_lost_home_occur', 'sack_yards_lost_yards':'sack_yards_lost_home_yards', 'penalty_occur':'penalty_home_occur', 'penalty_yards':'penalty_home_yards'})
    home_stat = home_stat.reset_index()
    home_stat = home_stat.drop(columns = ['index', 'team'])  

    stats_all = pd.concat([away_stat, home_stat], axis=1)
    
    # order columns to match model
    stats_all = stats_all[column_order]

    prediction_temp = model_obj.predict(stats_all)
    #prediction_value = str(prediction_temp[0])
    #predictions_list = predictions_list.append([prediction_value])
    probabilities_temp = pd.DataFrame(model_obj.predict_proba(stats_all), columns=['away', 'home'])
    probabilities_temp['id'] = game_id
    #probabilities_temp = probabilities_temp.rename(columns = {'away': away, 'home': home})
    # probabilities_df = probabilities_df.append(probabilities_temp)
    probabilities_df = pd.concat([probabilities_df, probabilities_temp], axis=0).reset_index(drop=True)

In [73]:
# probs_historical = probs_historical.append(probabilities_df)
probs_historical = pd.concat([probs_historical, probabilities_df], axis=0).reset_index(drop=True)
probs_historical.shape

(32, 3)

In [74]:
probs_historical

,away,home,id
0,0.723230,0.276770,401547609
1,0.417934,0.582066,401547610
2,0.747199,0.252801,401547611
3,0.494441,0.505559,401547612
4,0.810684,0.189316,401547613
5,0.569857,0.430143,401547614
6,0.239656,0.760344,401547616
7,0.680598,0.319402,401547617
8,0.907707,0.092293,401547618
9,0.574705,0.425295,401547615


In [75]:
probs_historical.to_csv('predictions/model_prediction_file.csv', index=False)